# GPU-accelerated image processing with ImageJ and CLIJ
This notebook illustrates how to use [CLIJ](https://clij.github.io/clij-docs/) to push images from ImageJ image variables to the GPU, process them there and pull results back to visualise them. You can find executable code examples using CLIJ online:
* [ImageJ macro](https://github.com/clij/clij/tree/master/src/main/macro)
* [ImageJ Jython](https://github.com/clij/clij/tree/master/src/main/jython)
* [Java](https://github.com/clij/clij/tree/master/src/main/java/net/haesleinhuepf/clij/demo)

Please note that notebook might not be the perfect environment for GPU-accelerated image processing. The speedup from using GPUs results from running longer workflows repeatedly in the GPU. This notebook rather serves explaining CLIJs application programming interface.

In order to run the notebook, please install [Anaconda](https://www.anaconda.com/) from its website. Afterwards, please install the [BeakerX](http://beakerx.com/) kernel from the Anaconda command line:
```
conda install -c conda-forge ipywidgets beakerx
```

The following section is only necessary for running CLIJ from notebooks.

In [1]:
// load imagej
%classpath config resolver imagej.public https://maven.imagej.net/content/groups/public
%classpath add mvn net.imagej imagej 2.0.0-rc-71
%classpath add mvn net.imagej imagej-notebook 0.7.1

// load clij
%classpath config resolver clij http://dl.bintray.com/haesleinhuepf/clij
%classpath add mvn net.haesleinhuepf clij_ 0.19.2

Added new repo: imagej.public


Added new repo: clij


## Getting started
The following code instantiates ImageJ. This is not be necessary if you run it from ImageJs script editor. 

In [2]:
// start ImageJ
ij = new net.imagej.ImageJ();

Apr 19, 2019 10:23:52 AM java.util.prefs.WindowsPreferences <init>


net.imagej.ImageJ@3e735b9f

Independent if you run this example code from the script editor or from a notebook, you need a `clij` variable giving you access to the GPU.

In [3]:
// start CLIJ
clij = net.haesleinhuepf.clij.CLIJ.getInstance();

net.haesleinhuepf.clij.CLIJ@5bd934a7

## Loading images and sending them to the GPU

In [4]:
// load an image
image = ij.io().open("https://imagej.net/images/blobs.gif");

[INFO] Verifying GIF format
[INFO] Reading dimensions
[INFO] Reading data blocks


In [5]:
// push images to the GPU
inputImage = clij.push(image);

// reserve memory in the GPU for the result image; same size and type as the input image
blurredImage = clij.create(inputImage);

ClearCLBuffer [mClearCLContext=ClearCLContext [device=ClearCLDevice [mClearCLPlatform=ClearCLPlatform [name=Intel(R) OpenCL], name=Intel(R) UHD Graphics 620]], mNativeType=UnsignedByte, mNumberOfChannels=1, mDimensions=[256, 254, 3], getMemAllocMode()=Best, getHostAccessType()=ReadWrite, getKernelAccessType()=ReadWrite, getBackend()=net.haesleinhuepf.clij.clearcl.backend.jocl.ClearCLBackendJOCL@6d850a5a, getPeerPointer()=net.haesleinhuepf.clij.clearcl.ClearCLPeerPointer@1eda1c93]

## Filtering images in the GPU

In [6]:
// apply a Gaussian blur
sigmaXY = 5; // in pixel units
sigmaZ = 0; // in pixel units
clij.op().blurFast(inputImage, blurredImage, sigmaXY, sigmaXY, sigmaZ);

true

In order to show an intermediate result from the script editor, you can call 

```
clij.show(blurredImage, "blurred image");
```

However, when working in Jupyter notebooks, you need to wrap or convert the result like this:

In [7]:
// show intermediate result
intermediateResult = clij.pull(blurredImage);

// the image needs to be wrapped/converted in order to be visible in notebooks.
import net.imglib2.img.display.imagej.ImageJFunctions;
ImageJFunctions.wrapReal(intermediateResult);

## Apply a threshold algorithm in the GPU

In [8]:
// reserve memory for another image in the GPU
binaryImage = clij.create(inputImage);

// apply the threshold algorithm
thresholdAlgorithmName = "Otsu";

clij.op().automaticThreshold(blurredImage, binaryImage, thresholdAlgorithmName);

true

In [9]:
// pull image back from GPU memory
resultBinary = clij.pull(binaryImage);

// wrap and show the image
import net.imglib2.img.display.imagej.ImageJFunctions;
ImageJFunctions.wrapReal(resultBinary);

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAD+CAYAAADVndu7AAAL+UlEQVR42u3dUYIdKQ5E0fT+Fz2zgZlu25UJitD54M+uR4LiAkISz69fv/7z0/Y8z4/bG/14u09f9u+rfrb0/6v+NrYfjTUA3DVGAACAq+M/oRPTDRMAAAAAglb/t40TAAAAAIJW/wQ4NQAgFVrE/xIANggMAAAAAELF9dM+xkwiABD/aQAkTe7k/gHAHAAkwQgAPupv7EQCQJ1tjAVA8tZuWp8AIGsHBgDhAKjdygHAsf5X2Uyj5x8AAKDBQVwJAKI/L34A2HGMBQAAqABAwtivBIDtPwA0ACA9TiEWAETPBwAAAKABQEX0JQAAAAAsBcA6HwDxA0Bq1aWm1f9aINDtAWuFS3ItgKTCKxMA8OZ3rgJA+w4jPaY+bUU92debmhgDgFMfCgCyLicB4Mvvj3ICbkvUUFrr17FV9CthnvqNqIIgk1d/EOhzvH757Te3/qN2AVMFyeiJ/+tx+Gospy+Sx3IBNqZqbo9TT90J3V68oh4GuX3ttyHegPh3FBi5rZfXi4ISwB2jEhi1GwB/uzt/rIDE5zq0227/8d81GYNacQAAAH/Y3yaDcPwAAABYCgAeeAAAgDIAnD6Du38HAAAIM4ytWW3E35MTcXXcGZ8QXDsAAGB0AAAAF77l+pgzOgAQBDQ/IxIArDjmI3guxj4yyuAAwG5s8ZgzNgDgkM0ZBwAAAHNTPAfHA98YGAA0zs+G7wYAq/+V8VIUpScBDgAAYOw9NvEDAAAsysJMD6qZWgB1FADSa8IzxHkQ2L4ojQdAQ7YW8e/yfST5PMYCoPlteK8T7VuJASDgg7aHnEqFzozdrwLAphdtAUAADgCErSbEDwC3S89VXgN61574AWBpIFDitprjy3Eo2R5HhQKnGpInyABgc9DTw5D2RJsBQEbtvqMP4Lhbn5H8AgAAcCMgCQCk3q6cM3YIANpSALDDIAAQKQBY/QFAAwCrPwBoIgE781MAAACufT8AfPM9Ecch28gMw0yBwPbVP21sAMA9NAC88E2pYwQAnFH1FYG+/q7EORwfCkz8ciMSxnvCeP1Ia9JrASAxRBYAXtKdc6TaexNLZL0ZB//VWN+ew1f05hyZd9/eXiX37W+ftvrfhtqnABBtZYc07Up02ur/VnzBWAA0niP5SHJrMkxc/U/vZo4DYFKQCwCozXfz3n81ACaEugKAoKi3F6OK7f9pAEw3NrH2/VGRJ0J+Y1b/bQC4JTAA6AulBQBht7EASH+sZXt2IQCExSVMAEDCLiQukSb5/n8CAKYYVRIAmus1bADAuMS7iYPXFnhzU3BpBVvbMw0B4CDp2wAwTfwn/AKcy9/GIDyt3t4mx9uNayqBSTN3tW8fa5/WLVMbAKaKHwCyjx9PwtalxesNANq048eTsAo2XXs1nf0BYDYMfvP/zBdk2713ZMAI8bdCI8Mj3Rb4QvxaNACmJ09sNXbiz50bAACACAgQc04i2qcAOJU/LQd/zzsNjbAaXvj1zCAnA2Bz8ZG0p8mmzWFA5edzg5uaPaX6UIff4tIWe/rYnxvY9gAc7w/MFv+0dxXWAeBm4Ydtzq4NAgGAQADczPve5ukmfs+tjwTA6aIg28uUt52NAWAgABqcZZvuorcELKU+tX6wBqMAEW2f+BMAcKhfAKABQDsA/uF3AEADgGYA/MvvAIAGAK1Xsb/xWwCgAUAjAH7ztwBAcwvQGItxFACMemddfACYGYD2B78JANselwCB8+M4+EFaAEgUAQCw2TEAIGSpyhKCYsfUYMrZ3wcBtvUSAIhZ2SsFTZcCgJh31etT1BQArubrM+59Rm2cBjoBXYPNXN0YNfF/DgDXYH0ASBtj4j8cCegKrPdsmzzOxP/CGCULwQ7g3OtJiePK6fwb8+wOfCcAbKHnzf2V3+bV3XcLMK1CjbP4RRC4AuuAQEuOupuLw+nKVr/8stgJBimycWjpsi0e8Nba+An36PIbBpcH3wAAj0fevXoV6zEXAgAwxPBSnktvOwNvz2N4GgblnwYn1fAul4kas+sAgI9vYwBgx513QrBR+nwkpjMDAACMFn/SnCQWNHnat0UAAACNVY0AAABWAWD6vKSWNXs2OEYAoAMAbSnKawAwIQmG+HOCgBLnCAAKACAZJQMAE+cKAEJWA+KfmwOQDAIACHMKqRF/ztA2QKAKAEMeLVSkoqTsGgAEXAOmGQSBZ41xOwRiAZC4LSTwTMCCwDMRzFlbQwLPLUIBAEUAOA0B4u44XjVDIPFdw2dYZwi/3LdiFzDi3O91YOK/WIjSjcCk3TYA3Dz/blr9twBg2nX3Z8+Dtz/UmeTjSBPPhhDhkBiMnQCYGAa7Ka5i2xsRg2MvACC9Ei4A8En9YGyJP/3xjVThEP+IMVW2Of0BjlThAMCI8bT63yg8AgCSw4aMIwCkA+C0z2FjABMAeLJpdDDOZgAQ/3MOAJuj5aZH4228Myf+5wwADNZ8ADQkVxH+Yf/JtmScdgA01T4k+gPOUwM3S/wMXTt6fWrwAGD7vK8V/0YAfOUsA4Ds+QYARvEjw9iW0Sbpq6TYiJVhRvhvalWbbTs9ACgHweaQ1u0Pdq4TPwCIaW+OxgOAQwDgwMoGgGo8S7f/PwEATzbxA8C5AKYxAHClBQAAcH5RHAEA99pdIlKV93xB1lgATE1waTubAUAWAJLKvv01ABJy3AFAUc6UcuyTdprPVC+xwpYA0Cj+m7uA//k3J55vbaH1PSH0N6Hoy7/91jNthbOCAkCb429y0ZdnmpFsf+ASADqu/G714Y//xiQjSTHEhsctG8Q//M29DGfppBUuxSABINv52vD7AHBpkL8UPwDkvCEAAC9/fIqQWgCQuI2dNs4AsAwAX4s/CQDiLX7Fi/84ANK3pU3iT0vumjre8TUTAGA3ABLSu6eP92nYRAKgwTHVDICNxTBO9HM6lJ9JYtwMAEIH3SsRjdPECADaVgBcCSuedue6EQCEDgDXXpRuS3lMjAEg9HvXri1FZT4DwI2rok1hwIS+GwC3v+P1kmDtASoAAAArAZAwMOroA8CGl5he9bm1GMDGhBQA2P0aUz0AJlcFJn4AaHAoextweTIIAAAAY1uWDLLRQbZh+w8AA4XVsqICAABottMAAADa9nM0APTMCwAQ/7rS660+DgDQ6hJpAGBALoBG/I0AaJojANBq78+lAV+qB6ABgHcMuq9nAQAAPAoSOGev/R7BEH97Jl3T/L3+G0QDABKbFtuIAQEAEPjZGEfbB9EAAPH/fHxj+040XYlEAJB9Pq8AgJXiniMLAO7O1XoAMJhvhQoAbhzGAoDR3E/LBQAAuAIAhjMjpJX4ASACANOiyFoSWwAAAI4DoCWRpO1+nvgB4EgcQNszXsRP/M2+rRUAmGTkAAAAY7+pEQDTDJ3wBQGN/aa21RYA/n/fiV+A2yf1AJq326mrv5gNDQAGQMB2XQOA8K22FF2t+Fgzw8ABAAC0SwB4pbxwMQASjwGMm+PxrwBwKw8dAABAOx95+Lz2yCAASNPV4qJXn1dfGi0WGQDkCGJrpujrAPjTPwoAcyAgdTo8S+/U8+CTylEBAACcGqdNNSKOAeB2uGmCwQAAh+mka+vn7T/augtIMnLi3/HM2Ut6/YQqAAAAkRmT2445z0dUEQ1Y0s9tADgxhpO+4/nwbFFzzm51AP1t/zaUTAMAAIgzhtP9ai+Y0rDbuQaAm8UU07bUyT6K9opJ6TudqwC4WXhii2e4oQ8AUAyAf/voLatqOzQBoBwAb3/89K31RO9463htKoa6GgCSSXqv4QAAANR1BwA3AY0AkKcNAFt3Aa4BhwEArAAAAA4GAtnKA8DmeIDVuQBb66QBAACkhwMfrwnIk78bArICM+tXPI3iB4DsuIltD6GOKQiiSioAbPSUT7Cda0VBGaU29bi18T2F42XBAU

## Multiplying pixel values in the GPU
The binary image contains values 0 and 1, which are hard to differentiate without being able to change brightness/contrast. Let's multiply the image with 255.

In [10]:
// reserve memory for another image in the GPU
multipliedBinaryImage = clij.create(binaryImage);

// multiply all pixels with 255
clij.op().multiplyImageAndScalar(binaryImage, multipliedBinaryImage, 255);

// pull result back from GPU
resultMultiplied = clij.pull(multipliedBinaryImage);

// wrap and show the image
import net.imglib2.img.display.imagej.ImageJFunctions;
ImageJFunctions.wrapReal(resultMultiplied);

<img src="data:image/png;charset=utf-8;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAD+CAYAAADVndu7AAAMQklEQVR42u2dy5LkKAxF/f8/PbObVU9XVhqk+ziKqF1FGoTuQcgYnn8O2PM8r/9O24k23WzfrXamtB8bGgMAsBuMAAAArPpfoRHqgQkAAAAAMJr9TwcnAAAAAMBo9neAUwIAXKGF+A8BoEFgAAAAAABTcb1to80gAgDEPw0Ap8FVbh8A0AGAE4wAwKX22g4kAIiLDVkAOKd2am0CAF4ZGAAwB0BsKgcAxtofFTOJlX8AAAASCsSRAMDmxQ8AOpaxAAAARADAwfeVACD9BwAJAHDfp2ALAIwaAAAAABgAiNh9CQAAAAAoBUBdDQDxA4DbbWf2X/C9CwBS4eJ8FsDp9qcD4GQ/qwCQnmG476l3m1En27qpCRkATHUUAPDVpRIAbvbfqgg41UmyAJ9K9Q3fKE1mapPjGgAaZpTmopqqP7ZjS3GCtAIAQY/4b/vhli/VJ8mxbwE2OpZgiH8+E9qevNb8pVqVb99vgPg7DhjZ1svxQ0ERwE5QtYOvHQDfZudPWiAgPsTfCoD/a8df/y8pGDafjwEAx8z1SQoIlh8AAACUAkAlCDEAAABM1+C8fwcAAMAsMBSDD9MSv/o3Eat+J/TYgksGAAAIOgAAABb6su5zgg4ApANAtT8SPifguNoaAMz2TcrnBBwAIBsr9jnBBgCSxyXNDwAAADA2wWMwvvGNAAMAiePT0G8AwOy/4i+HMWqJSwAAACJS0ZPj1BSTAAAARKSi6jBSjkkrALifCU8g6kGgfVKSB0DC11qIv6v24VTzkAVA8t3w3E7UNxMDAIMObUKAmkh2BgQAQmYTAHDXH8k+AAABswniBwDf9j+t+Pq0BhLiB4ZKANiqmT3KgeQAgeSiV4NfnOPxyDPbA4kryABA86anh0DSKAIBAMT/abuOPks5iFTOc0s2AKDdp9vPBwDl1jpmxCEAwEoBQBwaAQADAMz+AAADAMz+AABzgEBiPwEAAFgJUADgO/tbLIdIIz0C0wUC7bO/m28AgFFQAgBdALj6CABQjDo2jqk1jkSQHwVA0zoyvRjFa7+74r+ynfeN1hQBQOq/6xt2/fkA4LXeWEd6paOpH6X89HxFAGyP4RG9sY70En/qRyk3K+lqs/821K4CIHUdSYak4yOFeNwcx6P1NtaRXuKP3pV2qf+Ks/90NjMOgJtEBwD4aPu3AcChxrcYANjPGE/+bkT6Pw0A9WBzED8AuCcg9WL3lQ13CP++wADAbvqvCHUAYDBYqQBwv6ylNa4AgAmpFQHgkIXYfUjj/P5fAQAqQeUEAJmZY2lPuzsA1CaTR9F5aqm3MwDcDmxN/9IQAAySPg0AauKfqAtQXL4zrv/9Vmq1N6nwtvGaio1Jmlnt6WWtNACUKqZOtYhJ8QMA7+XH45C6pFS9AQCmtvx4HGbBFABMVY4BADD4xTcN+oJMAsBE0QjxY7+IFY+KdBIAPmkf4sekAaD+8URrsCN+37EBAADAAgKY3rJvBQBT308rCAEAcE2ZcuF3DQATJ6hsB1PqbJNwH4H6GBqc/DznXNevp5oBkFS3WEqx1X0/59j0DThpEEgTv9q9CnUA+K1TWtZhjTPPlvgBwDIAPnXM9oA0VZ6bxM916wIA+JujFAam8RVUypsKALAEgIRiWbop9XUbAOpXrQ+ewcgGEaxP/A4AGGoXAMAAQDoA/vIcAIABgGQA/PAcAIABALV+DrYBAGAAIBEAHz4LAGCdEFDt5/CzEb+DMABAx47AhWcCADcxAAG/mBW+kBYAOIoAABCzMgDA9gSAD4jZlz7Fme6Bjy+I2TUAYPsBj1+I2RUAYDpBjo+I21EAJFRKAQDZEn4zuCSDk3oI7hO+I9M8AACVQQUA909PTvIhy80XAFAVAuKfuUPR3Z/UnP7y5ywEMoC525Mc/Yp9MM7uIgAAM1e1YffHfuXZVHUzAOBWd6F4KQKCtuBPhcDEs91nQbexHmkfs5//sdgOAcnORtGjyxLWvp84KfVs/MnnTfYT8Q8dD94AAC6P3AGOUs2BpS4AkA48l+vS09bAaZmuJAA2v/JyDbzlY6Jksg4AcPltDADoeOftsNnIfTwcP2cGAABAWvxOY+J4oEkMAKhAA4AW8QMAAFAHAPVxcT3WLB4ALm8BAID32ACARQedfD7i13v16DBGACAAAKmWBgDFsQIAJrMB4tf9BsAZBADArCjEGfFzgdYAgSgAiFxayCEVAeIHACavAd0CAvPycToEbAHgmBZinoAFAo8imL1SQ2wmCAEAAJCCAJaxvEqGgOO9ho9YYxB+eG2FLEBi3c/twIh/8SBK3ggoZdsAYHP92zT7twDgdB8vt6UXAEk1DjfxNGwRNtmD0QkAxW2wLeJvAcB2sfva9eBtJ7dOBjgAYEk67FvE7375hqtwEL+ETzm22f0CDlfhAAAJfzL7bxw8AgD0quWF4gcACQCYrjkoQoDZfxAAjeI/df8gAMjKYAL8x3v/BAB88/y22gzifwEAnKUPgN+0wX2MsEPx9tMP4jQvAPzUFuexwi4UT3GclvgJdGwy1gAAAKgf91rxNwLgVrEMAHiPNwAgKF4FRtsXba3jHCX+ZgB84tCtwXEKtLZMDwCEg0BhcNLqIwBAVPwAQHOAHNsPAIoBQAHLGwCcxlOa/r8BAJVsxA8APnt2FAB4pQUAAMD8pCgBAN5rZ4mIU3nnD2S1BYDqBy5pazMA4AUAp2PfvgaAwzfuAEAjoNwAsOFHpUzzmXywqmMBQOfNPJs+3BizP/6m4vqWFJq2q4t/K05PP+tRm+GYQQFAWuFP+dCXRy1I2i+4BAAZr/y22vDr31AKEpdATLjcMkH84nfueRRLlWY4l4AEAN7F14TnA4AlJ98UPwCYaTcAEASAi5BSAOCYxqr5GQCUAeC2+J0A4Cz+TQCo2SgA3NPSJPFvVtCT3lrYn5kAALoB8GnflNu27e9p2FgCIKEwlQwAZXPx9/SGHxsApFSmEX+O+FOOMosCwMnnAgAAwAWpLwBwyvlbA96cigKA7PMLjn5cpPaqqHEPALYj/qRDZa4B4JsGutJwIyixbgBs9+P4kWDbTnELSgwAWADAwTFug4Z1A0ChL09KALSs2QBA7y7MOgB84ijH4MQAgEo/iMbQZQoAAAAAoHiZktBm0n8AYA+DlBkVAAAAjHQaAAAArH0dDQByxgUAIP66o9dTaxwAABsVEH3Q6dvXbUAmiL8VAEljBACw8bWzMwSSxurV85ELAOAeA9/xev1s5AIAuBTEb8yOPQ+5IP70L+mSxu/4M5AMAODDpuIYwR0AAAi887F1fCAZAID43/vXtu1IRndwAEDf+jwCAMwUe4UsALA7VvUAIGDuChUAZIA6EgAEzdwMDQAAgBQACJyd9HwTAIwbAHgVeG0FnAkxAgAAMA6AlA9JXMXPu38A8HVftoPbsdCG+Fm6OfulAgBKQQ4AAIBsnxIBoBboCD8fArZ9SpttAcD/tx3xnx9H+z6Rbmtfdb69YQgLhxoAuHzkEuk6BgC8U+2UJQCG/SE2NQIcAAAAbAkAR44XDgaA4zIAo/D4FQC2vkMHAAAA+328XAHAV5cMAgApCGD54j+UuR+8aTRYZADARxBNwj+QuZ/7UQCgA4HWmTDmK72p68GVjqMCAABgyk/JfV0DwNsOpQNgEgKIP+tyk1UAkAV4fR2I+DuuOTuk1ytUAQAAwEr8zpedvunLc4kq7AYMaWcbACZ8qNSP5+LaImadnVoA+rZ96QBIOR4OAJQFw3S7UsXvCP4Xn4Vffcdov8Z2SX+32pMMgITzIVcB8JsOqzpafR2c0AYAEAyAnzrdMqumQxMAhAPgdOfVU2vF6niqv5oOQ60GgMvrpUZrPDYNABQAYDOlBgC8CYgAAAYAWrMAXgOKAQBYAQAAMLgRiFQeADTvB1AG7nUAuA

## Cleaning up memory in the GPU
When working with GPUs, it is important to release memory afterwards:

In [11]:
inputImage.close();
blurredImage.close();
binaryImage.close();
multipliedBinaryImage.close();

null

Voila! When running this notebook, please try to stick to the top-bottom order, otherwise you might overwhelm your GPU. Just as an example, if you call a block with `clij.push(...);` again and again, there is more and more memory allocated in the GPU. Notebooks might not be the ideal environment for running GPU-accelerated code. This notebook rather exists for illustration / education.

Feedback is welcome: rhaase@mpi-cbg.de

Happy coding!

Cheers,
Robert